In [1]:
import numpy as np
from qutip import *
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import math
from numpy import *

In [2]:
from modules.spin_arch import *
from modules.su_algebra import *

In [3]:
from IPython.display import Latex

In [4]:
Ntrunc = 3 #global operator for truncation level or total number of dimensions

### Hamiltonian: 
$$
H' = \sum_{n,i} \left(\frac{(g^n_{i-1})^2 \mathcal H(i)}{v^n_{i-1} - \omega} + v^n_i - \Delta_i \right) \sigma^n_{i,i} +  \frac{1}{2} \sum_{n \neq m,i}\frac{g^n_{i}g^m_{i}}{\omega - v^m_{i}} ( \sigma^n_{i,i+1}\sigma^m_{i+1,i} + \sigma^n_{i+1,i}\sigma^m_{i,i+1} )
$$

where $\mathcal H(i) = 1$ for $i>0$, otherwise 0.

### Parameters:
$$v^n_{i-1} = (\sqrt{8 E^n_c E^n_j})i - 0.5*E^n_c i (i+1) $$
with default values set as $v \approx 60 * 100Mhz$, $g = 100Mhz$, $\omega = 55*100MHz$ 

In [5]:
#initialize system
N = 4
Nlevel = Ntrunc

Ej, Ec, g_arr, omega = generate_tmon_arch(N,Nlevel,o=59.87)
sys1 = tmon_system(N, Nlevel, Ec, Ej, g_arr, omega)

In [6]:
H = sys1.H_I()

59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952
59.87 0.0 -0.571067524025816
59.87 60.745553203367585 0.6155941838618952


In [7]:
sys1.freq_list()

[[0.0, 60.745553203367585, 118.99110640673517],
 [0.0, 60.745553203367585, 118.99110640673517],
 [0.0, 60.745553203367585, 118.99110640673517],
 [0.0, 60.745553203367585, 118.99110640673517]]

In [8]:
sys1.self_energy

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### Hamiltonian Evolution: $$e^{-iHt} \rho_0 e^{iHt}$$

In [9]:
psi_2 = basis(Nlevel,0) + sqrt(2)*basis(Nlevel,1) + basis(Nlevel,2)
psi_2 = psi_2/psi_2.norm()
psi_2 = psi_2 * psi_2.dag()
psi_2

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.25       0.35355339 0.25      ]
 [0.35355339 0.5        0.35355339]
 [0.25       0.35355339 0.25      ]]

### For pure states: $$F_{\psi} = 4 \Delta J^2_n = nCn^T \quad \quad C_{\alpha,\beta} = Cov<J_\alpha, J_\beta> = \frac{1}{2}[<J_\alpha J_\beta>+ <J_\beta J_\alpha>] - <J_\alpha><J_\beta>$$ where: $$<O> = Tr(\rho O) $$

In [10]:
# class hberg_evol:
#     def __init__(self,S,H):
#         self.S = S #value of n goes from 0 to N-1
#         self.H = H #cut off dimension/truncation
#         #This is different from d which is the level that comes into hamiltonian.
#         #d refers to the highest excitation present in the initialised state
#         #d  by default is set to (Nlevel-2) as (d+1) at max can be (Nlevel-1)
#         self.d = Nlevel-2
#         self.E_c= E_c
#         self.E_j = E_j
#         self.g_arr = g_arr #g^n_i = g_arr[n][i]
#         self.omega = omega #freq of photonic mode
#         self.v_arr = self.freq_list() #v[n][i] = v^n_i
#         self.self_energy = np.zeros((self.N,self.d + 1))
    
#     def freq_list(self):
#         #calculate v^n_i
#         #for |i><i|_n, coefficient is  \sqrt(8E^n_j*E^n_c)*i - E^n_c/2*i(i+1)
#         arr = []
#         for n in range(self.N):
#             temp_arr = []
#             for i in range(self.d+1):
#                 v_ni = np.sqrt(8*self.E_c[n]*self.E_j[n])*i - 0.5*self.E_c[n]*i*(i+1)
#                 temp_arr.append(v_ni)
#             arr.append(temp_arr) 
#         return arr

In [11]:
# Spin 1
# Sx=Qobj(ensemble_op(op_padded2(Qobj(np.array([[0,sqrt(2),0],[sqrt(2),0,sqrt(2)],[0,sqrt(2),0]])/2),Nlevel), N))
# Sy=Qobj(ensemble_op(op_padded2(Qobj(1j*np.array([[0,-sqrt(2),0],[sqrt(2),0,-sqrt(2)],[0,sqrt(2),0]])/2),Nlevel), N))
# Sz=Qobj(ensemble_op(op_padded2(Qobj(np.array([[1,0,0],[0,0,0],[0,0,-1]])),Nlevel), N))

In [12]:
# Spin 1
Sx=Qobj(ensemble_op(op_padded(Qobj(np.array([[0,1,0],[1,0,1],[0,1,0]])/np.sqrt(2)),Nlevel), N))
Sy=Qobj(ensemble_op(op_padded(Qobj(1j*np.array([[0,-1,0],[1,0,-1],[0,1,0]])/np.sqrt(2)),Nlevel), N))
Sz=Qobj(ensemble_op(op_padded(Qobj(np.array([[1,0,0],[0,0,0],[0,0,-1]])),Nlevel), N))

In [13]:
def hevol(t,S):
    tlis1=linspace(0,t,int(t/2)+10)
    options = Options()
#     print(options.ntraj)
    options.ntraj=1
    return mesolve(H,S,tlis1,[],[], options = options).states[-1]

In [14]:
eps=0
rho=tensor([psi_2]*N)
def opt_mat(t1=28,t2=71.5-28-eps,S=[Sx,Sy,Sz,Sx**2,Sy**2,Sz**2,(Sx*Sy+Sy*Sx)/2,(Sx*Sz+Sz*Sx)/2,(Sz*Sy+Sy*Sz)/2],l=3):
    Q=zeros([l,l],dtype=complex)
    M=zeros([l,l],dtype=complex)
    A1=[]
    B1=[]
    for i in tqdm(range(l)):
        A1.append(hevol(t1,S[i]))
        B1.append(hevol(t1+t2,S[i]))
    for i in tqdm(range(l)):
        for j in tqdm(range(l)):
            A=Qobj(A1[i])
            B=B1[j].copy()
            C=B1[i].copy()
            M[i][j] = 1j*expect(rho,(A*B-B*A))
            Q[i][j] = real((1/2)*expect(rho,(C*B+B*C))-expect(rho,(C))*expect(rho,(B)))
    return Q,M

In [15]:
190-86.285

103.715

In [16]:
Q,M=opt_mat()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
M1=M.copy()
Q1=Q.copy()

In [18]:
# Q1[0][0]=20
# Q

In [19]:
Qobj(Q)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 5.20417969e-02 -6.78721268e-02  8.82491703e-03]
 [-6.78721268e-02  2.00319644e+00  6.22404818e-05]
 [ 8.82491703e-03  6.22404818e-05  2.00000000e+00]]

In [26]:
(Qobj(Q).eigenstates()[0])**-1

array([20.14351423,  0.49999272,  0.49861252])

In [21]:


for i in range(len(M1)):
    for j in range(len(M1[0])):
        if abs(M1[i][j])<1e-3:
            M1[i][j]=0
        if abs(Q1[i][j])<1e-3 and i!=j:
            Q1[i][j]=0

In [27]:
Qobj(M)*Qobj((linalg.inv(Q)))*Qobj(M.transpose())#[0][0]
#.eigenstates()[0]
# Q1[8][7]

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[ 4.4796055 +7.11300407e-11j  0.23754374+7.11301678e-11j
   5.86479452+2.21304014e-11j]
 [ 0.23754374+7.07526627e-11j  1.23194716+1.08891046e-10j
  -0.10098953-3.33670637e-11j]
 [ 5.86479452+2.21308257e-11j -0.10098953-3.33643161e-11j
   7.92536476+0.00000000e+00j]]

In [23]:
Q2=(linalg.inv(Q1))

In [24]:
Qobj(Q2)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 2.01194209e+01  6.81684461e-01 -8.87761100e-02]
 [ 6.81684461e-01  5.22298938e-01 -3.00790440e-03]
 [-8.87761100e-02 -3.00790440e-03  5.00391721e-01]]

In [25]:
Q2[0][0]=Q2[3][3]=0

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
Q2=Qobj(Q2)

In [ ]:
# Qobj(M@linalg.inv(Q)@((M.transpose().conjugate())))
Q3=Qobj(M@Q2@((M.transpose().conjugate())))
# Qobj(M.transpose()@linalg.inv(Q)@M)

In [28]:
A=(Qobj(M)*Qobj((linalg.inv(Q)))*Qobj(M.transpose()))
# A=Qobj((M.transpose()@linalg.inv(Q)@M)
A=Qobj(A)
A

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[ 4.4796055 +7.11300407e-11j  0.23754374+7.11301678e-11j
   5.86479452+2.21304014e-11j]
 [ 0.23754374+7.07526627e-11j  1.23194716+1.08891046e-10j
  -0.10098953-3.33670637e-11j]
 [ 5.86479452+2.21308257e-11j -0.10098953-3.33643161e-11j
   7.92536476+0.00000000e+00j]]

In [29]:
eigenvalues, eigenvectors=A.eigenstates()

In [30]:
(eigenvalues)
eigenvectors[-1]

Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.59930799+4.20633115e-12j]
 [0.00555058+1.53699302e-12j]
 [0.8004993 +0.00000000e+00j]]

In [31]:
# newvector=[[array(eigenvectors[-1])[i][0] if i <3 else 0]  for i in range(9)]
newvector=Qobj(eigenvectors[-1])
newvector

Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[0.59930799+4.20633115e-12j]
 [0.00555058+1.53699302e-12j]
 [0.8004993 +0.00000000e+00j]]

In [32]:
Qobj((array((Qobj(linalg.inv(Q)*Qobj(M.transpose().conjugate()))*newvector)))/((Qobj(linalg.inv(Q)*Qobj(M.transpose().conjugate()))*newvector).norm()))

Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[-0.3184999 +7.46260352e-11j]
 [ 0.94677116+3.72628042e-12j]
 [ 0.04671375+0.00000000e+00j]]

In [ ]:
((array((Qobj(linalg.inv(Q)@M.transpose())*eigenvectors[-1])))/((Qobj(linalg.inv(Q)@M.transpose())*eigenvectors[-1]).norm()))

In [ ]:
0.17538138/0.9327553,0.04384733/0.23318348

In [ ]:
#Calculation of covariance matrix
# for now step (2,3) can be avoided as exact analytical results avaialble
# FI = n^T C n
def C_matrix(rho, J_arr):
    #do a dimension check for rho and J_arr[i] as well
    d = len(J_arr)
    C = np.zeros((d,d), dtype=complex)
    for i in range(d):
        for j in range(d):
            C[i][j] = 0.5*((rho*J_arr[i]*J_arr[j]).tr()+ (rho*J_arr[j]*J_arr[i]).tr())- (rho*J_arr[i]).tr()*(rho*J_arr[j]).tr()
    return C

def FI_max_cal(state_list, J_arr):
    FI_max_arr = []
    optimal_evec = []
    i = 0
    for state in tqdm(state_list):
        matrix = C_matrix(state, J_arr)
        eigenvalues, eigenvectors = np.linalg.eig(matrix)
        #temp_arr.append([str(i) + "time step: " + str(np.around(eigenvalues.real, 4))])
        #print(str(i) + "time step: " + str(np.around(eigenvalues.real, 4)))
        max_index = np.argmax(eigenvalues)
        max_eigenvalue = eigenvalues[max_index]
        max_eigenvector = eigenvectors[max_index]
        FI_max_arr.append(4*max_eigenvalue)
        optimal_evec.append(max_eigenvector)
        i = i+1
    
    return FI_max_arr, optimal_evec 
    

In [ ]:
# Checking expectation value for the J, spin one

def expvalcheck(state):
    
#     return np.sqrt(expect(state, op1)**2 + expect(state, op2)**2 + expect(state, op3)**2)
    return expect(state, op1)

In [ ]:
#Maximum Eigenvalue
#when n is the eigenvector with maximum eigen value of C, FI is maximized
# Define the matrix
FI_max_arr = []
optimal_evec = []
# Expectation value for J(spin half)
exp_check=[]
i = 0
#temp_arr = []
for state in tqdm(result.states):
    matrix = C_matrix(state, J_arr)
#     matrix = C_matrix(state, J_arr_spinhalf)
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    #temp_arr.append([str(i) + "time step: " + str(np.around(eigenvalues.real, 4))])
    #print(str(i) + "time step: " + str(np.around(eigenvalues.real, 4)))
    max_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_index]
    max_eigenvector = eigenvectors[max_index]
    FI_max_arr.append(4*max_eigenvalue)
    optimal_evec.append(max_eigenvector)
    exp_check.append(expvalcheck(state))
    i = i+1



In [ ]:
#Plot FI_max vs time/\theta
#plt.figure(figsize=(15,10))
plt.plot(times,FI_max_arr)
plt.plot(times,np.array(exp_check))
# plt.plot(times,exp_check)
plt.grid()
# plt.title(str(N) + " tmon system, spin 1/2, with "+str(len(J_arr))+ " tensor generators")
plt.title("Eigenstate of Jx for spin 1/2")
plt.xlabel("time")
plt.ylabel("F_max and $\\langle J^2 \\rangle$")
plt.legend(["FI","$\\langle J_x \\rangle$"])
plt.show()

In [ ]:
#Plot <J> vs time/\theta
#plt.figure(figsize=(15,10))
plt.plot(times,exp_check)
# plt.title(str(N) + " tmon system, spin 1/2, with "+str(len(J_arr))+ " tensor generators")
plt.grid()
plt.xlabel("time")
plt.ylabel("$\\langle \hat{J_x} \\rangle$")

In [ ]:
array(sigmay()).conjugate(),array(sigmay())

In [ ]:
help(Adjoint())

In [ ]:
linalg.inv(array([[1,0],[0,0]]))